# Lab 7: convolution layer
by Domrachev Ivan, B20-Ro-01

In [1]:
from nn_from_scratch.neurons import Convolution

import numpy as np
from matplotlib import pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers

2023-10-23 14:31:54.450532: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 14:31:54.512423: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-23 14:31:54.513203: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-23 14:31:55.562584: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Part 1. One layer NN

The sample convolutional layer was implemented, which supports the convolution provided in the lecture
> Note: the solution is far from being generalized, f.e. it lacks padding, stride settings, as well as support of batches of the pictures

In [2]:
input_dim = (3, 10, 10)
conv_dim = (3, 2, 2)
conv = Convolution(input_dim, conv_dim)
output_dim = conv._output_dim

# Random input values (x itself, and assumed partial derivative)
x_input = np.random.random(input_dim)
dL_dy = np.random.random(output_dim)

output = conv.forward(x_input)
dL_dx = conv.backward(dL_dy)
dL_dw = conv._W_pd

## Part 2. Validation via `Keras`

In [52]:
x_input_batched = tf.constant(
    np.moveaxis(
        np.expand_dims(x_input, axis=0),
        1, -1
    ), 
    dtype=tf.float32
)
weights_reshaped = tf.constant(
    np.moveaxis(
        conv.W,
        0, -1
    ), 
    dtype=tf.float32
)

In [53]:
conv_keras = layers.Conv2D(
    1, 2,
    input_shape=x_input_batched.shape[1:],
    use_bias=False,
    kernel_initializer=tf.keras.initializers.Constant(weights_reshaped)
)

In [91]:
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x_input_batched)  # Watch the input tensor for gradient computation
    conv_output = conv_keras(x_input_batched)
    print(conv_output.shape)
conv_output_np = conv_output.numpy().transpose(0, 3, 1, 2).squeeze()

(1, 9, 9, 1)


(9, 9)

1. Check that weights are loaded correctly

In [77]:
np.all(conv_keras.weights[0].numpy().squeeze() == weights_reshaped)

True

2. Check that layer output is correct

In [121]:
np.all((conv_output_np - output[0]) < 1e5)

True

In [99]:
dL_dy_keras = tf.constant(np.expand_dims(dL_dy, axis=-1), dtype=tf.float32)

dL_dx_keras = tape.gradient(
    conv_output, x_input_batched, output_gradients=dL_dy_keras
)
dL_dw_keras = tape.gradient(
    conv_output, conv_keras.trainable_variables, output_gradients=dL_dy_keras
)

dL_dx_keras_np = np.moveaxis(dL_dx_keras[0].numpy().squeeze(), -1, 0)
dL_dw_keras_np = np.moveaxis(dL_dw_keras[0].numpy().squeeze(), -1, 0)

3. Check that backpropogation for both input and weights is correct

In [120]:
np.all((dL_dx_keras_np - dL_dx) < 1e-5)

True

In [119]:
np.all((dL_dw_keras_np - dL_dw) < 1e-5)

True